In [ ]:
# !pip install pandas
# !pip install seaborn
# !pip install sklearn
# !pip install pyenchant
# !apt-get install libenchant1c2a

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing import sequence
import os 
# os.chdir('/content/drive/MyDrive/NeuralRes

# os.chdir('/content/drive/MyDrive/MSCA_31009/Final_Project/')earch/wine-net/')

def remove_punctuation(s):
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  for ele in s:
      if ele in punc:
          s = s.replace(ele, "")
  return s

def clean_description(review, swap):
  terms = review.split()
  for x in terms:
    try: x = swap[('t_' + str(x))]
    except KeyError: x = ''
  return ' '.join(terms)


In [ ]:
wines = pd.read_csv('winemag-data-130k-v2.csv')
wines = wines.drop(columns = ['Unnamed: 0'])

wines2 = pd.read_csv('winemag-data_first150k.csv')
wines2.head()

# Intersect and append two review tables
columns = [value for value in wines.columns if value in wines2.columns] # intersection(wines.columns, wines2.columns)
wines = wines[columns]
wines2 = wines2[columns]
wines = pd.concat([wines, wines2]).drop(columns = ['designation', 'winery'])

wines = wines.sample(n=20000)

# Imputation steps
median_price = wines.price.median()
median_points = wines.points.median()

# Impute variables
wines.price = wines.price.fillna(median_price)
wines.points = wines.points.fillna(median_points)
wines = wines.fillna('')
wines.price = wines.price.astype(int)
wines.description = wines.description.apply(lambda x: remove_punctuation(x.lower()))

Y = wines.variety
output_shape = Y.nunique()
Y0_price = wines.price
Y1_points = wines.points.apply(lambda x: 1 if x > 90 else 0)
wines = wines.drop(columns=['variety', 'price', 'points', 'region_1', 'region_2'])

wines.head()

In [ ]:
text = wines.description
tfidf = TfidfVectorizer() 
review_vector = tfidf.fit_transform(text)
words = tfidf.get_feature_names_out()
rev_array = review_vector.toarray()
words_df = pd.DataFrame(rev_array, columns = words)

words.shape

In [ ]:
import enchant

enchant.list_languages()
eng = enchant.Dict("en_US")
drops = []
for column in words_df.columns:
    english = eng.check(column)
    if any(map(str.isdigit, column)): 
        drops.append(column)
    if not(english):
        drops.append(column)

words_df = words_df.drop(columns = drops)
words_df.head()

In [ ]:
# vectorizing the rest of the data in the table.
wines.head()

In [ ]:
from keras.preprocessing.text import Tokenizer
X = wines.description.to_numpy()
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)
X_processed

In [ ]:
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
print(Y1_points.nunique())
encoder = LabelEncoder()
encoder.fit(Y)
Y_processed = encoder.transform(Y)

In [ ]:
# words_df = words_df.reset_index()
# wines = wines.reset_index()
# wines = pd.concat([wines, words_df], axis=1)
# wines.shape
# wines = wines.drop(columns = ['index'])

## Variety Classifier

In [ ]:
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

X_train, X_test, y_train, y_test = train_test_split(X_processed, Y_processed, test_size=0.25, random_state=0)

input_dim = X_train.shape[1]

## RNN Build

In [ ]:
# from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
# from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
# from tensorflow.keras import regularizers
# from tensorflow.keras import backend as K

# model = Sequential()
# model.add(Embedding(input_dim=X_train.shape[0], output_dim=X_train.shape[1], weights=[X_train], input_length=max_log_length, trainable=False))
# model.add(SpatialDropout1D(0.5))
# model.add(Conv1D(filters=32, kernel_size=10 , kernel_regularizer=regularizers.l2(0.00001), padding='same'))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Bidirectional(LSTM(units=64,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
# model.add(SpatialDropout1D(0.5))
# model.add(Conv1D(filters=32, kernel_size=10 ,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Bidirectional(LSTM(lstm_units,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
# model.add(SpatialDropout1D(0.5))
# model.add(Conv1D(filters=32, kernel_size=10 ,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
# model.add(LeakyReLU(alpha=0.2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Bidirectional(LSTM(lstm_units,dropout=0.5, recurrent_dropout=0.5)))
# model.add(Dense(4,activation='softmax'))

## CNN Build

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

model = Sequential()
model.add(Embedding(input_dim=X_train.shape[0], output_dim=X_train.shape[1], weights=[X_train], input_length=max_log_length, trainable=False))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=64, kernel_size=4 , kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=64, kernel_size=4 ,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=64, kernel_size=4 ,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))

model.add(Dense(1,activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()


In [ ]:
model.evaluate(X_test, y_test, batch_size=128)